In [316]:
import pandas as pd 
import numpy as np
from bokeh.io import output_notebook
from dotenv import load_dotenv
import mysql.connector
import os 
import json
pd.set_option("display.max_rows", None, "display.max_columns", None)

load_dotenv()
api_key = os.environ['api_key']

output_notebook()
bokeh_width, bokeh_height = 1000, 800



Loading BokehJS ...

In [317]:

df_lora = pd.read_csv(r'data/Lora.csv')

print(min(df_lora['RSSI']))
print(max(df_lora['RSSI']))

df_lte = pd.read_csv(r'data/LTE_M.csv')
df_lte['RSRP'] = df_lte['RSRP'].apply(lambda x: x if x < 255 else np.nan)
df_lte['RSRP'] = df_lte['RSRP']-141 # -141 since 1 = -140

df_lte['RSRQ'] = df_lte['RSRQ'].apply(lambda x: x if x < 255 else np.nan)
df_lte['RSRQ']  = df_lte['RSRQ'].apply(lambda x: -20 if x == 0 else -20+(x/2)) #convert RSRQ and -20 because 1 = -19.5 


df_lte['RSSI'] = df_lte['RSSI'].apply(lambda x: x if x < 99 else np.nan)
df_lte['RSSI'] = df_lte['RSSI'].apply(lambda x: -113 if x == 0 else -113+(x*2))

df_nbiot = pd.read_csv(r'data/NB_IoT.csv')
df_nbiot['RSRP'] = df_nbiot['RSRP'].apply(lambda x: x if x < 255 else np.nan)
df_nbiot['RSRP'] = df_nbiot['RSRP']-141 # -141 since 1 = -140

df_nbiot['RSRQ'] = df_nbiot['RSRQ'].apply(lambda x: x if x < 255 else np.nan)
df_nbiot['RSRQ']  = df_nbiot['RSRQ'].apply(lambda x: -20 if x == 0 else -20+(x/2))

df_nbiot = df_nbiot[df_nbiot['RSRQ'].notna()]


df_sigfox_db = pd.read_csv(r'data/sigfoxDbClean.csv')
df_sigfox_backend = pd.read_csv(r'data/sigfoxBackendClean.csv', sep = ';')

df_sigfox_backend.columns=["Data","RSSI1","RSSI2","RSSI3","RSSI4","Sequence number backend"]

df_sigfox_backend['SessionID'] = df_sigfox_backend['Data'].str[:2].str.upper()
df_sigfox_backend['SequenceNumber'] = df_sigfox_backend['Data'].str[-4:].astype(int)

print(min(df_sigfox_backend['RSSI1']))
print(max(df_sigfox_backend['RSSI1']))

#print(df_sigfox_db)
#print(df_sigfox_backend)

-120.0
-84.0
-122.0
-89.0


In [318]:
from bokeh.io import show
from bokeh.plotting import gmap
from bokeh.models import GMapOptions
from bokeh.models import ColumnDataSource
from bokeh.models import HoverTool, TapTool
from bokeh.models import Title
from bokeh.io import export_png
from selenium import webdriver
my_driver = webdriver.Firefox(executable_path=r'INSERTPATH')

from bokeh.transform import linear_cmap
from bokeh.palettes import Magma256 as palette
from bokeh.models import ColorBar


In [319]:

#df_merged_sigfox = df_sigfox_db.merge(df_sigfox_backend, on = ['SequenceNumber', 'SessionID'])
df_merged_sigfox = df_merged_sigfox.rename(columns={'Sequence number backend': 'Sequence_number_backend'})
#print(df_merged_sigfox['Sequence_number_backend'])


In [320]:
def plotSigfox(lat, lng, zoom=14, map_type='terrain'):

    json_style = '[ { "featureType": "all", "elementType": "labels.icon", "stylers": [ { "visibility": "off" } ] }]'
    gmap_options = GMapOptions(lat=lat, lng=lng, map_type=map_type, zoom=zoom, tilt = 0, styles = (json_style))

    p = gmap(api_key, gmap_options, title='Sigfox coverage', width=bokeh_width, height=bokeh_height,  toolbar_location = 'below')
    p.title.align = "center"
    p.title.text_font_size = "30px"

    data_source = ColumnDataSource(df_merged_sigfox[['Technology','Latitude','Longitude','RSSI1', 'Sequence_number_backend']])

    mapper = linear_cmap('RSSI1', palette, -135, -80) # from https://support.sigfox.com/docs/link-quality:-general-knowledge

    my_hover = HoverTool()

    my_hover.tooltips = [('Latitude', '@Latitude'), ('Longitude','@Longitude'), ('RSSI (dBm)', '@RSSI1'), ('Sequence Number (Sigfox)', '@Sequence_number_backend')]

    center = p.circle('Longitude','Latitude', size = 10, alpha = 0.75, color = mapper, source=data_source)

    legend = ColorBar(color_mapper = mapper['transform'], location = (0,0), major_label_text_font_size  = "18px")
    p.add_layout(legend, 'right')
    p.add_layout(Title(text="Latitude", align="center", text_font_size="25px"), "below")
    p.add_layout(Title(text="Longitude", align="center", text_font_size="25px"), "left")
    p.add_layout(Title(text="RSSI (dBm)", align="center", text_font_size="25px"), "right")
    p.xaxis.major_label_text_font_size = "18px"
    p.yaxis.major_label_text_font_size = "18px"
    p.add_tools(my_hover)
    export_png(p, filename="sigfoxPlotFinal.png", webdriver = my_driver)
    show(p)
    
    return p

In [328]:
def plotLora(lat, lng, zoom=14, map_type='terrain'):

    json_style = '[ { "featureType": "all", "elementType": "labels.icon", "stylers": [ { "visibility": "off" } ] }]'
    gmap_options = GMapOptions(lat=lat, lng=lng, map_type=map_type, zoom=zoom, tilt = 0, styles = (json_style))

    p = gmap(api_key, gmap_options, title='LoRa coverage', width=bokeh_width, height=bokeh_height,  toolbar_location = 'below')
    p.title.align = "center"
    p.title.text_font_size = "30px"

    data_source = ColumnDataSource(df_lora[['Technology','Latitude','Longitude','Margin','NOfGateways','RSSI','SNR']])

    mapper = linear_cmap('RSSI', palette, -123, -80) # from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5038744/

    my_hover = HoverTool()

    my_hover.tooltips = [('Latitude', '@Latitude'), ('Longitude','@Longitude'),
                         ('Number of Gateways', '@NOfGateways'), ('RSSI (dBm)', '@RSSI'), ('SNR (dB)', '@SNR')]

    center = p.circle('Longitude','Latitude', size = 10, alpha = 0.75, color = mapper, source=data_source)

    legend = ColorBar(color_mapper = mapper['transform'], location = (0,0), major_label_text_font_size  = "18px")
    p.add_layout(legend, 'right')
    p.add_layout(Title(text="Latitude", align="center", text_font_size="25px"), "below")
    p.add_layout(Title(text="Longitude", align="center", text_font_size="25px"), "left")
    p.add_layout(Title(text="RSSI (dBm)", align="center", text_font_size="25px"), "right")
    p.xaxis.major_label_text_font_size = "18px"
    p.yaxis.major_label_text_font_size = "18px"
    p.add_tools(my_hover)
    export_png(p, filename="loraPlotFinal.png", webdriver = my_driver)
    show(p)
    
    return p

In [322]:
def plotLte(lat, lng, zoom=14, map_type='terrain'):

    json_style = '[ { "featureType": "all", "elementType": "labels.icon", "stylers": [ { "visibility": "off" } ] }]'
    gmap_options = GMapOptions(lat=lat, lng=lng, map_type=map_type, zoom=zoom, tilt = 0, styles = (json_style))

    p = gmap(api_key, gmap_options, title='LTE-M coverage', width=bokeh_width, height=bokeh_height, toolbar_location = 'below')
    p.title.align = "center"
    p.title.text_font_size = "30px"

    data_source = ColumnDataSource(df_lte[['Technology','Latitude','Longitude','RSRQ','RSRP','RSSI','Network_Operator']])

    mapper = linear_cmap('RSRP', palette, -140, -44) # from SARA r410 doc

    my_hover = HoverTool()
    
    my_hover.tooltips = [('Latitude', '@Latitude'), ('Longitude','@Longitude'),
                         ('RSRP (dBm)', '@RSRP'), ('RSRQ (dB)', '@RSRQ'), ('RSSI (dBm)', '@RSSI'), ('Operator', '@Network_Operator')]
    center = p.circle('Longitude','Latitude', size = 10, alpha = 0.75, color = mapper, source=data_source, legend_group='Network_Operator')
    p.legend.location = "top_right"
    p.legend.click_policy="hide"
    p.legend.title = "Network Operator"

    legend = ColorBar(color_mapper = mapper['transform'], location = (0,0), major_label_text_font_size  = "18px")
    p.add_layout(legend, 'right')
    p.add_layout(Title(text="Latitude", align="center", text_font_size="25px"), "below")
    p.add_layout(Title(text="Longitude", align="center", text_font_size="25px"), "left")
    p.add_layout(Title(text="RSRP (dBm)", align="center", text_font_size="25px"), "right")
    p.xaxis.major_label_text_font_size = "18px"
    p.yaxis.major_label_text_font_size = "18px"
    p.add_tools(my_hover)
    export_png(p, filename="ltePlotFinal.png", webdriver = my_driver)
    show(p)
    
    return p

In [323]:
def plotNbiot(lat, lng, zoom=14, map_type='terrain'):

    json_style = '[ { "featureType": "all", "elementType": "labels.icon", "stylers": [ { "visibility": "off" } ] }]'
    gmap_options = GMapOptions(lat=lat, lng=lng, map_type=map_type, zoom=zoom, tilt = 0, styles = (json_style))

    p = gmap(api_key, gmap_options, title='NB-IoT coverage', width=bokeh_width, height=bokeh_height, toolbar_location = 'below')
    p.title.align = "center"
    p.title.text_font_size = "30px"

    data_source = ColumnDataSource(df_nbiot[['Technology','Latitude','Longitude','RSRQ','RSRP','RSSI','Network_Operator']])

    mapper = linear_cmap('RSRP', palette, -140, -44) # from SARA r410 doc

    my_hover = HoverTool()

    my_hover.tooltips = [('Latitude', '@Latitude'), ('Longitude','@Longitude'),
                         ('RSRP (dBm)', '@RSRP'), ('RSRQ (dB)', '@RSRQ'), ('RSSI (dBm)', '@RSSI'), ('Operator', '@Network_Operator')]

    center = p.circle('Longitude','Latitude', size = 10, alpha = 0.75, color = mapper, source=data_source, legend_group='Network_Operator')
    p.legend.location = "top_right"
    p.legend.click_policy="hide"
    p.legend.title = "Network Operator"

    legend = ColorBar(color_mapper = mapper['transform'], location = (0,0), major_label_text_font_size  = "18px")
    p.add_layout(legend, 'right')
    p.add_layout(Title(text="Latitude", align="center", text_font_size="25px"), "below")
    p.add_layout(Title(text="Longitude", align="center", text_font_size="25px"), "left")
    p.add_layout(Title(text="RSRP (dBm)", align="center", text_font_size="25px"), "right")
    p.xaxis.major_label_text_font_size = "18px"
    p.yaxis.major_label_text_font_size = "18px"
    p.add_tools(my_hover)
    
    export_png(p, filename="nbiotPlotFinal.png", webdriver = my_driver)
    show(p)
    
    return p